# Genetic algorithms


Genetic algorithms are inspired in evolution.

Each solution is called a chromosome, and a group of them is called population. The same way in biology different organism evolve to survive, in genetic algorithms (GA from now) solutions evolve to fit the problem. This evolution is based on the Darwininan "survival of the fittest" approach, where individuals best fitted for the problem will hopefully help to create a new population even more suited for the problem than the previous one.

Some of them can even mutate to offer a new adaptation to the problem.

This process happens for generations until the condition is met.

## Steps 

GA follow these steps.

1. __Generate__ a random population
2. __Evaluate__ the fitness of each chromosome in the population
3. Select two parent chromosomes from the population (the better fitness, the better chances to be chosen)
4. __Crossover__: With a crossover probability, crossover the parents to create a new chromosome. If the condition is not met, the offspring is an exact copy of the parents.
5. __Mutation__: With a mutation probability, change one of the values of a chromosome.
6. Add the new individuals to the new population.
7. __Evaluate__ the new population.
8. If the final condition is met, stop and return the best chromosome in the population.
9. If the condition is not met, repeat from step 3.

## Enconding

The chromosomes should represent information about the solution they are trying to find. There are 3 main types.

#### Binary Encoding:

Binary is the easiest and most common. In this encoding, every chromosome is a array of bits, 0 or 1.

| Chromosome | Values |
| ---------- | --------- |
| A| 00100101010010 |
| B | 01010110000100 |

Example of use: Knapsack problem. A true / false array of elements that should be chosen for the knapsack.

#### Value Enconding:

In this case the array is composed by different values, as we try to optimize said values.

| Chromosome | Values |
| ---------- | --------- |
| A| 10 29 0.7 45 8 |
| B | A B A Z X M Y |

Example of use: Parametrization of a function


#### Ordering Encoding:

The array represents the optimal ordenation for a given problem.

| Chromosome | Values |
| ---------- | --------- |
| A| 4 3 2 5 1|
| B | 2 3 1 5 4 |

Example of use: Travelling salesman.

#### Tree encoding:

This case is mainly used for genetic programming. Each chromosome is a tree of elements, steps or objects.

     Chromosome A                       Chromosome B
     [(X+1)*A +Z]                       [Z*A + BAC]
          [A]                               [B] 
          / \                               / \  
        [*] [+]                           [*] [+] 
        /     \                           / \   \
      [X+1]   [Z]                       [A] [C] [*]
                                                / \
                                              [A] [Z]
                                              
Example of use: Finding a function for a given values

## Example

In this case, we will create a GA that can modify himself into the phrase we ask for.

The GeneticAlgorithm class implements the basic steps explained before. 

In [13]:
import random

class GeneticAlgorithm(object):
    def __init__(self, genetics):
        self.genetics = genetics
        
    def run(self):
        population = self.genetics.initial()
        while True:
            fits_pops = [(self.genetics.fitness(ch),  ch) for ch in population]
            if self.genetics.check_stop(fits_pops):
                break
            population = self.next(fits_pops)
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size = len(fits)
        nexts = []
        while len(nexts) < size:
            parents = next(parents_generator)
            cross = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents
            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
        return nexts[0:size]

The Genome class implements the proper methods for this kind of encoding (value encoding).

In [14]:
class Genome(GeneticAlgorithm):
    def __init__(self, target_text,
        limit=200, size=300,
        prob_crossover=0.9, prob_mutation=0.2):
        self.target = self.text2chromo(target_text)
        self.counter = 0
        self.limit = limit
        self.size = size
        self.prob_crossover = prob_crossover
        self.prob_mutation = prob_mutation
        print("target string[%s]"%target_text)
        print("in numeric: [%r]"%self.text2chromo(target_text))
    
    def probability_crossover(self):
        return self.prob_crossover
    
    def probability_mutation(self):
        return self.prob_mutation

    def initial(self):
        return [self.random_chromo() for j in range(self.size)]

    def fitness(self, chromo):
        return -sum(abs(c - t) for c, t in zip(chromo, self.target))

    def check_stop(self, fits_populations):
        self.counter += 1
        if self.counter % 10 == 0:
            best_match = list(sorted(fits_populations))[-1][1]
            fits = [f for f, ch in fits_populations]
            best = max(fits)
            worst = min(fits)
            ave = sum(fits) / len(fits)
            print(
                "[G %3d] score=(%4d, %4d, %4d): %r" %
                (self.counter, best, ave, worst,
                self.chromo2text(best_match)))
            print("in 1:255 values: %r"%(best_match))
            return self.counter >= self.limit

    def parents(self, fits_populations):
        while True:
            father = self.tournament(fits_populations)
            mother = self.tournament(fits_populations)
            yield (father, mother)

    def crossover(self, parents):
        father, mother = parents
        index1 = random.randint(1, len(self.target) - 2)
        index2 = random.randint(1, len(self.target) - 2)
        if index1 > index2: index1, index2 = index2, index1
        child1 = father[:index1] + mother[index1:index2] + father[index2:]
        child2 = mother[:index1] + father[index1:index2] + mother[index2:]
        return (child1, child2)

    def mutation(self, chromosome):
        index = random.randint(0, len(self.target) - 1)
        vary = random.randint(-5, 5)
        mutated = list(chromosome)
        mutated[index] += vary
        return mutated

    def tournament(self, fits_populations):
        alicef, alice = self.select_random(fits_populations)
        bobf, bob = self.select_random(fits_populations)
        return alice if alicef > bobf else bob

    def select_random(self, fits_populations):
        return fits_populations[random.randint(0, len(fits_populations)-1)]

    def text2chromo(self, text):
        return [ord(ch) for ch in text]
    
    def chromo2text(self, chromo):
        return "".join(chr(max(1, min(ch, 255))) for ch in chromo)

    def random_chromo(self):
        return [random.randint(1, 255) for i in range(len(self.target))]    

Now, we choose a phrase and we'll see how the GA evolves until it transforms itself into the same phrase.

In [15]:
if __name__ == "__main__":
    GeneticAlgorithm(Genome("carlos dime una frase")).run()

target string[carlos dime una frase]
in numeric: [[99, 97, 114, 108, 111, 115, 32, 100, 105, 109, 101, 32, 117, 110, 97, 32, 102, 114, 97, 115, 101]]
[G  10] score=(-488, -787, -1147): '{^\x88o\x87tl@\x81jzTh\x87R\x1d8fX\xaaz'
in 1:255 values: [123, 94, 136, 111, 135, 116, 108, 64, 129, 106, 122, 84, 104, 135, 82, 29, 56, 102, 88, 170, 122]
[G  20] score=(-258, -373, -556): '{^\x7f\x83qt T`jZ*rtR\x1ds_Y\xaaz'
in 1:255 values: [123, 94, 127, 131, 113, 116, 32, 84, 96, 106, 90, 42, 114, 116, 82, 29, 115, 95, 89, 170, 122]
[G  30] score=(-210, -239, -283): '{^\x7foqt Y`dh*utR#nfX\xa7v'
in 1:255 values: [123, 94, 127, 111, 113, 116, 32, 89, 96, 100, 104, 42, 117, 116, 82, 35, 110, 102, 88, 167, 118]
[G  40] score=(-171, -194, -212): '{b\x7foqt Yeng*roV#in\\\xa7v'
in 1:255 values: [123, 98, 127, 111, 113, 116, 32, 89, 101, 110, 103, 42, 114, 111, 86, 35, 105, 110, 92, 167, 118]
[G  50] score=(-138, -159, -179): 'vb|kqt!^inc%uo[\x1fin\\\xa7v'
in 1:255 values: [118, 98, 124, 107, 113, 116, 33

target string[carlos dime una frase]
in numeric: [[99, 97, 114, 108, 111, 115, 32, 100, 105, 109, 101, 32, 117, 110, 97, 32, 102, 114, 97, 115, 101]]
[G  10] score=(-331, -683, -1104): '[N\x85qds\x19jtRkv}\x96\\ \\u_r\x9e'
in 1:255 values: [91, 78, 133, 113, 100, 115, 25, 106, 116, 82, 107, 118, 125, 150, 92, 32, 92, 117, 95, 114, 158]
[G  20] score=(-188, -264, -426): 'Di\x85qls\x0bbhb_\x11\x95m] \\u_qq'
in 1:255 values: [68, 105, 133, 113, 108, 115, 11, 98, 104, 98, 95, 17, 149, 109, 93, 32, 92, 117, 95, 113, 113]
[G  30] score=(-125, -147, -179): 'dffqjs\x19bhh_\x1d}n\\ mu_dD'
in 1:255 values: [100, 102, 102, 113, 106, 115, 25, 98, 104, 104, 95, 29, 125, 110, 92, 32, 109, 117, 95, 100, 68]
[G  40] score=( -89, -109, -129): 'd]\x83mos\x1fbhlf\x1dwn] eq`dD'
in 1:255 values: [100, 93, 131, 109, 111, 115, 31, 98, 104, 108, 102, 29, 119, 110, 93, 32, 101, 113, 96, 100, 68]
[G  50] score=( -73,  -84,  -97): 'G`fmos\x1fbjlc\x1dwn` eq_qq'
in 1:255 values: [71, 96, 102, 109, 111, 115, 31, 98

_Adapted from https://gist.github.com/bellbind/741853 and http://www.obitko.com/tutorials/genetic-algorithms/ga-basic-description.php _ 